In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
#config_file = "configs/vgg_ssd300_voc0712_tdt4265_server.yaml"
config_file = "configs/train_waymo.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2020-04-13 13:42:38,563 SSD INFO: Loaded configuration file configs/train_waymo.yaml
2020-04-13 13:42:38,566 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'improved'
        #NAME: 'basic'
        PRETRAINED: True
        #OUT_CHANNELS: [128, 128, 256, 128, 128, 64, 64]
        #OUT_CHANNELS: [128, 256, 512, 256, 128, 64] # basic
        #OUT_CHANNELS: [128, 256, 512, 512] # resnet18
        OUT_CHANNELS: [128, 256, 512, 128, 128, 64] #resnet34
        INPUT_CHANNELS: 3
INPUT:
    IMAGE_SIZE: [320, 240]
DATASETS:
    TRAIN: ("waymo_train",)
    TEST: ("waymo_val", )
SOLVER:
    MAX_ITER: 30000
    LR_STEPS: [80000, 100000]
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 1e-3
OUTPUT_DIR: 'outputs/improved'
DATASET_DIR: "datasets"
2020-04-13 13:42:38,568 SSD INFO: Running with config:
DATASETS:
  TEST: ('waymo_val',)
  TRAIN: ('waymo_train',)
DATASET_DIR: datasets
DATA_LOADER:
  NUM_WORKERS: 16
  PIN_MEMORY: True
EVAL_STEP: 500
INPUT:
  IMAGE_SIZE: [320, 240]
  PIXEL_MEAN: [

In [4]:
model = start_train(cfg)

improved
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace

KeyboardInterrupt: 

In [ ]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)